## Пример запуска персонального AI-ассистента с NaiveRetriever-алгоритмом поиска

In [1]:
import sys
from tqdm import tqdm

# TO CHANGE
BASEDIR = "../.."
sys.path.insert(0, BASEDIR)

In [ ]:
TRIALS = 2
FIX_FILE_PATH = "./import_fix.py"
for _ in range(TRIALS):
    try:
      from src import PersonalAI, PersonalAIConfig, QAPipelineConfig, MemPipelineConfig, \
            GraphModelConfig, EmbeddingsModelConfig, EmbedderModelConfig

      from src.db_drivers import GraphDriverConfig, VectorDriverConfig
      from src.db_drivers.graph_driver import DEFAULT_INMEMORYGRAPH_CONFIG
      from src.db_drivers.vector_driver import VectorDBConnectionConfig

      from src.pipelines.qa.kg_reasoning import KnowledgeGraphReasonerConfig
      from src.pipelines.qa.kg_reasoning.weak_reasoner.knowledge_retriever import NaiveGraphSearchConfig
      from src.pipelines.qa.kg_reasoning.weak_reasoner import KnowledgeRetrieverConfig, QALLMGeneratorConfig, WeakKGReasonerConfig

      from src.pipelines.memorize import LLMExtractorConfig, LLMUpdatorConfig

      from src.utils import NodeType, Logger
    except RuntimeError as e:
        from pathlib import Path
        fix_path = Path(FIX_FILE_PATH)
        if fix_path.is_file():
            %run {fix_path} --base_dir BASEDIR
        else:
            raise e

/home/dzigen/Desktop/PersonalAI/.pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Задаём конфигурацию графа знаний

Для создания объекта класса PersonalAI (верхнеуровневый класс персонального ассистента) необходимо выполнить конфигурацию трёх основных его компонент:
* Графа знаний, который будет выступать в роли памяти ассистента и хранить поступающую информацию.
* QA-конвейера, с помощью которого ассистент будет отвечать на user-вопросы.
* Memorize-конвейера, с помощью которого ассистент будет добавлять новую информацию в память и выполнять её актуализацию.

Граф знаний состоит из двух моделей хранения информации: векторной и графовой. Соответственно нам потребуется инициализировать два конфигурационных data-класса: `GraphModelConfig` и `EmbeddingModelConfig`.

В конфиге для графовой модели потребуется указать параметры подключения к одной из доступных графовых бд.

In [3]:
# Graph model configuration
GRAPH_STORAGE_CONFIG = GraphDriverConfig(db_vendor='inmemory_graph', db_config=DEFAULT_INMEMORYGRAPH_CONFIG)
GRAPH_MODEL_CONFIG = GraphModelConfig(driver_config=GRAPH_STORAGE_CONFIG)

В конфиге для векторной модели потребуется отдельно указать параметры подключения к двум векторным бд, где будут храниться эмбеддинги триплетов и вершин из графовой модели соответственно.

In [4]:
# Vector model configuration
NODES_DB_PATH = f'{BASEDIR}/data/graph_structures/vectorized_nodes/testing'
TRIPLETS_DB_PATH = f'{BASEDIR}/data/graph_structures/vectorized_triplets/testing'
NEED_TO_CLEAR = True

VECTOR_NODES_STORAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(path=NODES_DB_PATH, need_to_clear=NEED_TO_CLEAR))
VECTOR_TRIPLETS_STIRAGE_CONFIG = VectorDriverConfig(db_config=VectorDBConnectionConfig(path=TRIPLETS_DB_PATH, need_to_clear=NEED_TO_CLEAR))

Также потребуется указать параметры для инициализации класса, с помощью которого будет выполняться перевод текста в его векторное представления (полечение эмбеддингов).

In [5]:
DEVICE = 'cuda'
EMBEDDER_MODEL_PATH = f'{BASEDIR}/models/intfloat/multilingual-e5-small'
EMBEDDER_MODEL_CONFIG = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH, device=DEVICE)

In [6]:
VECTOR_MODEL_CONFIG = EmbeddingsModelConfig(
    nodesdb_driver_config=VECTOR_NODES_STORAGE_CONFIG,
    tripletsdb_driver_config=VECTOR_TRIPLETS_STIRAGE_CONFIG,
    embedder_config=EMBEDDER_MODEL_CONFIG)

Далее нам потребуется сконфигурировать QA- и Memorize-пайплайны.

В рамках QA-конфигурации мы задаём алгоритм поиска, с помощью которого будет извлекаться информации из графа знаний.

In [7]:
NAIVE_RETRIEVER_CONFIG = NaiveGraphSearchConfig(max_k=50)

In [8]:
RETRIEVER_NAME = 'naive_retriever'
RETRIEVER_HYPERP = NAIVE_RETRIEVER_CONFIG

QUERY_PARSER_CONFIG = None
KNOWLEDGE_COMPARATOR_CONFIG = None
TRIPLETS_FILTER_NAME = None
TRIPLETS_FILTER_CONFIG = None

Для QAGenerator-стадии мы указываем язык, который будет использоваться для генерации необходимой информации c помощью LLM-агента. В рамках библиотеки прддерживается два языка: русский и английский.

In [9]:
LANGUAGE = 'en' # 'ru' | 'en' | 'auto'

In [10]:
# QA-pipeline configuration
QA_PIPELINE_CONFIG = QAPipelineConfig(
    reasoner_config=KnowledgeGraphReasonerConfig(
        reasoner_name='weak',
        reasoner_hyperparameters=WeakKGReasonerConfig(
            query_parser_config=QUERY_PARSER_CONFIG,
            knowledge_comparator_config=KNOWLEDGE_COMPARATOR_CONFIG,
            knowledge_retriever_config=KnowledgeRetrieverConfig(
                retriever_method=RETRIEVER_NAME,retriever_config=RETRIEVER_HYPERP,
                filter_method=TRIPLETS_FILTER_NAME, filter_config=TRIPLETS_FILTER_CONFIG),
            answer_generator_config=QALLMGeneratorConfig(lang=LANGUAGE))))

В рамках Memorize-конфигурации мы задаём только язык обрабатываемого текста, чтобы на шаге запуска LLM-агента выбрать болен оптимальные промпты и функцию парсинга ответа.

In [11]:
# Memorize-pipeline configuration
DELETE_OBSOLETE_INFO = False

MEM_PIPELINE_CONFIG = MemPipelineConfig(
    extractor_config=LLMExtractorConfig(lang=LANGUAGE),
    updator_config=LLMUpdatorConfig(lang=LANGUAGE, delete_obsolete_info=DELETE_OBSOLETE_INFO))

Таким образом, у нас была получена конфигурация гиперпараметров для инициализации персонального ассистена. 

In [12]:
PERSONALAI_CONFIG = PersonalAIConfig(
    graph_struct_config=GRAPH_MODEL_CONFIG,
    embedds_struct_config=VECTOR_MODEL_CONFIG,
    qa_pipeline_config=QA_PIPELINE_CONFIG,
    mem_pipeline_config=MEM_PIPELINE_CONFIG)

### 2. Инициализируем персонального ассистента

In [13]:
personalai = PersonalAI(config=PERSONALAI_CONFIG)

No sentence-transformers model found with name ../../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [14]:
personalai.kg_model.count_items()

{'graph_info': {'triplets': 0, 'nodes': 0},
 'embeddings_info': {'nodes': 0, 'triplets': 0}}

### 3. Пример работы Memorize-конвейера

Подготавливаем набор текстов на естественном языке (в нашем случае на английском) для их последующего сохранения в память ассистента.

In [15]:
messages = [
    "Mikhail Menshchikov is currently a second-year master's student at ITMO.",
    "Mikhail Menshchikov is studying in the Master's program 'Deep Learning and Generative AI'",
    "Mikhail Menshchikov completed his bachelor's degree at Petrozavodsk State University",
    "Petrozavodsk State University is where Mikhail Menshchikov received his bachelor's degree.",
    "Mikhail Menshchikov studied at Petrozavodsk State University and received a bachelor's degree."]
properties = [dict() for _ in range(len(messages))]

С помощью метода update_memory сохраняем сформированный набор информации в память.

In [16]:
for text, prop in tqdm(list(zip(messages, properties))):
    _, info = personalai.update_memory(text, prop)

100%|██████████| 5/5 [00:11<00:00,  2.40s/it]


### 4. Пример работы QA-конвейера

Теперь мы можем задавать вопросы ассистенту по имеющимся у него знаниях в памяти и получать ответы.

In [17]:
answer, info = personalai.answer_question("What program is Mikhail Menshchikov studying for his master's degree?")
print(answer)

Deep Learning and Generative AI


In [18]:
answer, info = personalai.answer_question("Where did Mikhail Menshchikov receive his bachelor's degree?")
print(answer)

Petrozavodsk State University
